In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder\
                    .appName('movie')\
                    .master('local')\
                    .getOrCreate()

23/12/10 10:06:58 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.18 instead (on interface wlp0s20f3)
23/12/10 10:06:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/10 10:06:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# define the schema for u.data
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True),
    StructField("_corrupt_record", StringType(), True)
])

opt = {'header':False, 'sep':'\t', 'mode':'permissive', 'inferschema':False}
data_df = spark.read.format('csv')\
                    .options(**opt)\
                    .schema(schema)\
                    .load("/home/chaitanya/GIT/P-SPARK/case_study1/u.data")
data_df.show(5)
data_df.count()
data_df.printSchema()
data_df.columns

+-------+--------+------+---------+---------------+
|user_id|movie_id|rating|timestamp|_corrupt_record|
+-------+--------+------+---------+---------------+
|    196|     242|     3|881250949|           null|
|    186|     302|     3|891717742|           null|
|     22|     377|     1|878887116|           null|
|    244|      51|     2|880606923|           null|
|    166|     346|     1|886397596|           null|
+-------+--------+------+---------+---------------+
only showing top 5 rows

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- _corrupt_record: string (nullable = true)



['user_id', 'movie_id', 'rating', 'timestamp', '_corrupt_record']

In [6]:

#define the schema for u.user
user_schema_field = [StructField('UserId', IntegerType(), True),
                     StructField('Age', IntegerType(), True),
                     StructField('Gender',StringType(), True),
                     StructField('Occupation',StringType(),True),
                     StructField('ZipCode', IntegerType(), True),
                     StructField('_corrupt_record',StringType(), True)]

user_schema = StructType(user_schema_field)

u_opt= {'header':False, 'sep':'|', 'mode':'permissive'}
user_df = spark.read.format('csv')\
                    .options(**u_opt)\
                    .schema(user_schema)\
                    .load('/home/chaitanya/GIT/P-SPARK/case_study1/u.user')

user_df.show(5)                 
user_df.count()
user_df.printSchema()

+------+---+------+----------+-------+---------------+
|UserId|Age|Gender|Occupation|ZipCode|_corrupt_record|
+------+---+------+----------+-------+---------------+
|     1| 24|     M|technician|  85711|           null|
|     2| 53|     F|     other|  94043|           null|
|     3| 23|     M|    writer|  32067|           null|
|     4| 24|     M|technician|  43537|           null|
|     5| 33|     F|     other|  15213|           null|
+------+---+------+----------+-------+---------------+
only showing top 5 rows

root
 |-- UserId: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [8]:
gender_df = user_df.groupBy(col('Gender')).count()
gender_df.show()

+------+-----+
|Gender|count|
+------+-----+
|     F|  273|
|     M|  670|
+------+-----+



In [14]:
# 2 Give the top 5 movies which are reviewed maximum number of times 
from pyspark.sql.window import Window


data_groupby_df = data_df.groupBy(col('movie_id')).count()
window_movie = Window.orderBy(col('movie_id').desc())
review_df = data_groupby_df.withColumn('ranking', dense_rank().over(window_movie))
review_df.show(5)

23/12/10 01:16:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+-----+-------+
|movie_id|count|ranking|
+--------+-----+-------+
|    1682|    1|      1|
|    1681|    1|      2|
|    1680|    1|      3|
|    1679|    1|      4|
|    1678|    1|      5|
+--------+-----+-------+
only showing top 5 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, dense_rank

# Create a Spark session
spark = SparkSession.builder \
    .appName('example') \
    .getOrCreate()

# Assuming data_df1 and data_df2 are two DataFrames with a column named 'count'
# You can adjust this according to your actual DataFrames

# Define a Window specification to rank by the 'count' column from data_df1
window_spec_df1 = Window.orderBy(col('data_df1_count').desc())

# Calculate the count of reviews for each movie in data_df1
movie_review_counts_df1 = data_df1.groupBy('movie_id').count().withColumnRenamed('count', 'data_df1_count')

# Add a dense rank column based on the review counts and the specified window specification
ranked_movies_df1 = movie_review_counts_df1.withColumn('rank_df1', dense_rank().over(window_spec_df1))

# Define a Window specification to rank by the 'count' column from data_df2
window_spec_df2 = Window.orderBy(col('data_df2_count').desc())

# Calculate the count of reviews for each movie in data_df2
movie_review_counts_df2 = data_df2.groupBy('movie_id').count().withColumnRenamed('count', 'data_df2_count')

# Add a dense rank column based on the review counts and the specified window specification
ranked_movies_df2 = movie_review_counts_df2.withColumn('rank_df2', dense_rank().over(window_spec_df2))

# Show the top 5 movies with the highest review counts for both DataFrames
top_movies_df1 = ranked_movies_df1.filter(col('rank_df1') <= 5)
top_movies_df2 = ranked_movies_df2.filter(col('rank_df2') <= 5)

top_movies_df1.show()
top_movies_df2.show()
